In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.gray();
from matplotlib.pyplot import imshow


import skimage
from skimage.io import imread, imsave
from skimage.color import rgb2gray, rgb2xyz, rgb2hsv, rgb2lab, rgb2yuv, rgb2ycbcr, rgb2ypbpr, ycbcr2rgb, rgb2rgbcie
from skimage.transform import resize

import cv2

import numpy as np


from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display



# ステレオマッチング

OpenCV BSD
https://github.com/opencv/opencv/blob/master/samples/python/stereo_match.py

In [ ]:
imL = imread('aloeL.jpg')
imR = imread('aloeR.jpg')


@interact(shift=(0,300,5))
def g(shift=0):
    
    fig = plt.figure(figsize=(15,5))

    ax = fig.add_subplot(1,3,1)
    imshow(imL)
    plt.axis('off')
    plt.title('left image')

    ax = fig.add_subplot(1,3,2)
    imshow(imR)
    plt.axis('off')
    plt.title('right image')


    ax = fig.add_subplot(1,3,3)
    imshow((np.roll(imR, shift, axis=1).astype(float)+imL.astype(float))/2 / 255)
    plt.axis('off')

    plt.show()

In [ ]:
imgL = cv2.imread('aloeL.jpg')
imgR = cv2.imread('aloeR.jpg')

window_size = 3
min_disp = 16
num_disp = 112-min_disp

In [ ]:
@interact(window_size=(1, 10, 1),
          min_disp=(0, 100, 1),
          num_disp=(16, 16*20, 16)
         )
def g(window_size=3, min_disp=16, num_disp=96):
    

    stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
                                    numDisparities = num_disp,
                                    blockSize = 16,
                                    P1 = 8*3*window_size**2,
                                    P2 = 32*3*window_size**2,
                                    disp12MaxDiff = 1,
                                    uniquenessRatio = 10,
                                    speckleWindowSize = 100,
                                    speckleRange = 32
    )

    # 視差の計算
    disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

    imshow(disp, vmin=0, vmax=200)
    plt.colorbar()

# 色空間の変換

In [ ]:
def plot_rgb2(im, rgb2xxx, tospace_str, cmap=['gray', 'gray', 'gray']):
    xxx = rgb2xxx(im) if rgb2xxx is not None else im

    fig = plt.figure(figsize=(15,3))

    for i, c in zip(range(3), tospace_str):
        ax = fig.add_subplot(1, 3, i + 1)
        imshow(xxx[:, :, i], cmap=cmap[i])
        plt.colorbar()
        plt.title('{0} channel min {1:.2f} max {2:.2f}'.format(c, xxx[:, :, i].min(), xxx[:, :, i].max() ))

    plt.show();    



im = imread('girl.jpg')

imshow(im)
plt.title("original RGB image")
plt.show()

plot_rgb2(im, None, 'RGB')
plot_rgb2(im, rgb2rgbcie, 'RGB')
plot_rgb2(im, rgb2xyz, 'XYZ')
plot_rgb2(im, rgb2lab, 'Lab')
plot_rgb2(im, rgb2yuv, 'YUV')
plot_rgb2(im, rgb2ycbcr, ['Y', 'Cb', 'Cr'])
plot_rgb2(im, rgb2ypbpr, ['Y', 'Pb', 'Pr'])


plot_rgb2(im, None, 'RGB', ['Reds_r', 'Greens_r', 'Blues_r'])
plot_rgb2(im, rgb2lab, 'Lab', ['gray', 'RdBu_r', 'gnuplot2'])
plot_rgb2(im, rgb2hsv, 'HSV', ['hsv', 'gray', 'gray'])

# YUV444, YUV422, YUV420

In [ ]:
im = imread('girl.jpg')


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 2, 1)
imshow(im)
plt.title("original RGB image")
ax = fig.add_subplot(1, 2, 2)
imshow(im[300:400, 550:650])
plt.show()




ycbcr = rgb2ycbcr(im)
h, w, _ = ycbcr.shape

#####

Y = ycbcr[:, :, 0]
Cb = ycbcr[:, :, 1]
Cr = ycbcr[:, :, 2]

fig = plt.figure(figsize=(15,3))

ax = fig.add_subplot(1, 3, 1)
imshow(Y)
plt.title('Y')

ax = fig.add_subplot(1, 3, 2)
imshow(Cb)
plt.title('Cb')

ax = fig.add_subplot(1, 3, 3)
imshow(Cr)
plt.title('Cr')

plt.show();   


YUV444 = (ycbcr2rgb(ycbcr) * 255).astype(np.uint8)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 2, 1)
imshow(YUV444)
plt.title("YUV444")
ax = fig.add_subplot(1, 2, 2)
imshow(YUV444[300:400, 550:650])
plt.show()


#####

Y = ycbcr[:, :, 0]
Cb = resize(ycbcr[:, :, 1], (h, w//2))
Cr = resize(ycbcr[:, :, 2], (h, w//2))

fig = plt.figure(figsize=(15,3))

ax = fig.add_subplot(1, 3, 1)
imshow(Y)
plt.title('Y')

ax = fig.add_subplot(1, 3, 2)
imshow(Cb)
plt.title('Cb')

ax = fig.add_subplot(1, 3, 3)
imshow(Cr)
plt.title('Cr')

plt.show();   

ycbcr[:, :, 1] = resize(Cb, (h, w))
ycbcr[:, :, 2] = resize(Cr, (h, w))

YUV422 = (ycbcr2rgb(ycbcr) * 255).astype(np.uint8)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 2, 1)
imshow(YUV422)
plt.title("YUV422")
ax = fig.add_subplot(1, 2, 2)
imshow(YUV422[300:400, 550:650])
plt.show()


#####

Y = ycbcr[:, :, 0]
Cb = resize(ycbcr[:, :, 1], (h//2, w//2))
Cr = resize(ycbcr[:, :, 2], (h//2, w//2))

fig = plt.figure(figsize=(15,3))

ax = fig.add_subplot(1, 3, 1)
imshow(Y)
plt.title('Y')

ax = fig.add_subplot(1, 3, 2)
imshow(Cb)
plt.title('Cb')

ax = fig.add_subplot(1, 3, 3)
imshow(Cr)
plt.title('Cr')

plt.show();   

ycbcr[:, :, 1] = resize(Cb, (h, w))
ycbcr[:, :, 2] = resize(Cr, (h, w))

YUV420 = (ycbcr2rgb(ycbcr) * 255).astype(np.uint8)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 2, 1)
imshow(YUV420)
plt.title("YUV420")
ax = fig.add_subplot(1, 2, 2)
imshow(YUV420[300:400, 550:650])
plt.show()




## 色の解像度を落として再構成

In [ ]:
im = imread('girl.jpg')


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 2, 1)
imshow(im)
plt.title("original RGB image")
ax = fig.add_subplot(1, 2, 2)
imshow(im[300:400, 550:650])
plt.show()


@interact(rate=(1, 50, 1))
def g(rate=2):

    
    ycbcr = rgb2ycbcr(im)
    h, w, _ = ycbcr.shape


    Y = ycbcr[:, :, 0]
    Cb = resize(ycbcr[:, :, 1], (h//rate, w//rate))
    Cr = resize(ycbcr[:, :, 2], (h//rate, w//rate))


    fig = plt.figure(figsize=(20,10))

    ax = fig.add_subplot(2, 3, 1)
    imshow(Y)
    plt.title('Y')

    ax = fig.add_subplot(2, 3, 2)
    imshow(Cb)
    plt.title('Cb')

    ax = fig.add_subplot(2, 3, 3)
    imshow(Cr)
    plt.title('Cr')


    ycbcr[:, :, 1] = resize(Cb, (h, w))
    ycbcr[:, :, 2] = resize(Cr, (h, w))

    YUV = (ycbcr2rgb(ycbcr) * 255).astype(np.uint8)

    ax = fig.add_subplot(2, 3, 4)
    imshow(YUV)
    plt.title("YUV with 1/{} reduction".format(rate))

    ax = fig.add_subplot(2, 3, 5)
    imshow(YUV[300:400, 550:650])

    plt.show()